# WMS services

In this notebook we verify that WMS services are working for all forecast variables

In [30]:
import ipyleaflet
import ipywidgets
import sidecar
import sqlmodel

from IPython.display import display

from arpav_ppcv import (
    config,
    database,
    datadownloads,
    operations,
)

settings = config.get_settings()
session = sqlmodel.Session(database.get_engine(settings))

all_covs = database.collect_all_coverages(session)

In [6]:
covs = {c.identifier: c for c in all_covs}

In [28]:
cov_id = "tasmin_30yr_anomaly_seasonal_agree_model_ensemble-30yr-forecast-model_ensemble-tasmin-anomaly-rcp85-tw1-lower_bound-winter"

In [17]:
arpav_grid = datadownloads.CoverageDownloadGrid.from_config(settings.coverage_download_settings.spatial_grid)
map_center = arpav_grid.shapely_box.centroid
m = ipyleaflet.Map(center=(map_center.y, map_center.x), zoom=7)
m.add_control(ipyleaflet.LayersControl(position="topright"))

In [29]:
sc = sidecar.Sidecar(title="map")

with sc:
    display(m)

In [19]:
wms_layer = ipyleaflet.WMSLayer(
    name=cov_id,
    url=f"http://localhost:8877/api/v2/coverages/wms/{cov_id}",
    layers=covs[cov_id].configuration.wms_main_layer_name,
    format="image/png",
    transparent=True,
)
m.add_layer(wms_layer)